Importing cellular automata & optimization classes, and other stuff

In [2]:
#Original Version.
#%pip install tables

import os
import sys
import shutil
import time

from typing import List, Type, Callable, Dict
from numpy import int32
from numpy._typing import NDArray
import importlib

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

#from EditedCode.blender import Lattice, clear_initial
from EditedCode.ruleset import conway, seeds, RuleSet
from EditedCode.genetic import Optimizer, Mutator, RulesetMutator, ArbitraryRulesetMutator, MutationSet
from EditedCode.objectives import surface_to_vol

import numpy as np
import pandas as pd

import time

Setting up optimizer and data logging code

In [6]:
def log_mutation(data_list: List[Dict], mutations: List[MutationSet], objective_val: float):
    """
    Given the data list reference, the mutation set, and the objective value after applying it, add it to the data logging list
    """
    ic_cell_pos = []
    ic_state_old = []
    ic_state_new = []
    srt_cell_pos = []
    srt_state_old = []
    srt_state_new = []

    for ic_mut in mutations.ic_mutations:
        ic_cell_pos.append(ic_mut.cell_pos)
        ic_state_old.append(ic_mut.old_state)
        ic_state_new.append(ic_mut.new_state)
    for srt_mut in mutations.srt_mutations:
        srt_cell_pos.append(srt_mut.cell_pos)
        srt_state_old.append(srt_mut.old_state)
        srt_state_new.append(srt_mut.new_state)
    
    data_list.append({
        "ic_cell_pos": np.array(ic_cell_pos), 
        "ic_state_old": np.array(ic_state_old), 
        "ic_state_new": np.array(ic_state_new), 
        "srt_cell_pos": np.array(srt_cell_pos), 
        "srt_state_old": np.array(srt_state_old), 
        "srt_state_new": np.array(srt_state_new), 
        "objective": objective_val,
    })

def run_experiment(iters: int, grid_sz: int, ruleset_mutator_class: Type[Mutator], rule_set: List[RuleSet], opt_func: Callable[[NDArray[int32]], int],
                   srt_num_mutate: int, ic_num_mutate: int, rule_mutate_prob: float):
    """
    Runs an experiment with the below hyperparameters:

    :param iters: The number of iterations the mutation algorithm (updating both IC and SRT) is going to run for
    :param grid_sz: The size of the square grid that we're going to update each iteration
    :param ruleset_mutator_class: The class of the SRT mutator we're going to use for this experiment
    :param rule_set: The set of rules that the SRT initially has (picked at random for each cell, then scrambled by the mutator)
    :param opt_func: The functions that gives the performance metric we're going to optimize
    :param srt_num_mutate: The number of SRT cells for which we're going to mutate the rule applied, each iteration
    :param ic_num_mutate: The number of IC cells for which we're going to mutate the rule applied, each iteration
    :param rule_mutate_prob: The probability, for each neighbor state tensor of the rule of a cell that's selected to be mutated, the final state is mutated
    """
    # TODO: separate SRT and IC mutations to have a certain number of each
    # TODO: add a flag to enable doing only SRT or only IC mutations in an iteration (in optimizer step, and then propagate into mutator)
    ruleset_mutator = ruleset_mutator_class(rules=rule_set, grid_size=grid_sz, mutate_p=1/(grid_sz**2) * (srt_num_mutate+ic_num_mutate), rule_mutate_p=rule_mutate_prob)

    optim = Optimizer(mutator=ruleset_mutator, objective=lambda grid: opt_func(grid))

    """
    Pandas Dataframe used to log experiment data is:

    ic_cell_pos (np.array) | ic_state_old (np.array) | ic_state_new (np.array) | srt_cell_pos (np.array) | srt_state_old (np.array) | srt_state_new (np.array) | objective (float)
    
    etc.

    initial state for IC is in entry 0 in ic_state_old, and SRT is in entry 0 in srt_state_old

    ic and srt mutation cell positions and states can have an extra dimension in the beginning to indicate they are batch updates
    """

    init_state = optim.state
    
    data_list = [{"ic_cell_pos": grid_sz, 
                  "ic_state_old": init_state.initial, 
                  "ic_state_new": None, 
                  "srt_cell_pos": -1, 
                  "srt_state_old": init_state.rules, 
                  "srt_state_new": None, 
                  "objective": 0}]

    for it in range(iters):
        if (it%5 == 0):
            print(f"Iteration {it}")
        # print(f"On iteration {it+1}...")
        accepted, new, old, mutations = optim.step()

        # data logging
        log_mutation(data_list, mutations, optim.objvalue)

        #if accepted:
        #    print("Got a better state!", optim.objvalue)

    # print(data_list)
    df = pd.DataFrame(data_list)
    # print(df)
    return df

timelogs = []

def repeat_experiment(experiment_name: str, num_expers: int, *args):
    """
    Perform (sequentially) multiple experiments that return a Pandas DataFrame and save all the data

    :param experiment_name: The name of the experiment to save the file
    :param num_expers: Number of times to run the experiment (and save all the data in one file)
    :param *args: The arguments to be passed to the experiment function
    """
    for i in range(num_expers):
        init = time.time()
        print(f'REPETITION {i}')
        ret_data = run_experiment(*args)
        timelogs.append(time.time() - init)
        print(f"Finished rep {i} in {time.time() - init}s")
        ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


Setting up experiments and gathering data

In [7]:
ITERATIONS_SET = [100]
GRID_SIZE_SET = [32]
NUM_REPEAT = 50
EXPERIMENT_NAME = "test_experiment"

for iters in ITERATIONS_SET:
    for grid_sz in GRID_SIZE_SET:
        print(f"RUNNING EXPERIMENT {EXPERIMENT_NAME} WITH {iters} ITERATIONS AND {grid_sz} SIZE GRID")
        repeat_experiment(f"{EXPERIMENT_NAME}_{iters}ITERS_{grid_sz}GRID", NUM_REPEAT, iters, grid_sz, ArbitraryRulesetMutator, [conway(), seeds()], surface_to_vol, 10, 10, 2/3)

RUNNING EXPERIMENT test_experiment WITH 100 ITERATIONS AND 32 SIZE GRID
REPETITION 0
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Iteration 55
Iteration 60
Iteration 65
Iteration 70
Iteration 75
Iteration 80
Iteration 85
Iteration 90
Iteration 95
Finished rep 0 in 75.20939755439758s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_9868\974963337.py:106: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new'],
      dtype='object')]

  ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


REPETITION 1
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Iteration 55
Iteration 60
Iteration 65
Iteration 70
Iteration 75
Iteration 80
Iteration 85
Iteration 90
Iteration 95
Finished rep 1 in 75.69101405143738s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_9868\974963337.py:106: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new'],
      dtype='object')]

  ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


REPETITION 2
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Iteration 55
Iteration 60
Iteration 65
Iteration 70
Iteration 75
Iteration 80
Iteration 85
Iteration 90
Iteration 95
Finished rep 2 in 74.95247769355774s


...

C:\Users\LAMM\AppData\Local\Temp\ipykernel_9868\974963337.py:106: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new'],
      dtype='object')]

  ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


REPETITION 48
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Iteration 55
Iteration 60
Iteration 65
Iteration 70
Iteration 75
Iteration 80
Iteration 85
Iteration 90
Iteration 95
Finished rep 48 in 74.61784434318542s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_9868\974963337.py:106: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new'],
      dtype='object')]

  ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


REPETITION 49
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Iteration 25
Iteration 30
Iteration 35
Iteration 40
Iteration 45
Iteration 50
Iteration 55
Iteration 60
Iteration 65
Iteration 70
Iteration 75
Iteration 80
Iteration 85
Iteration 90
Iteration 95
Finished rep 49 in 74.63368892669678s


C:\Users\LAMM\AppData\Local\Temp\ipykernel_9868\974963337.py:106: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['ic_cell_pos', 'ic_state_old', 'ic_state_new', 'srt_cell_pos',
       'srt_state_old', 'srt_state_new'],
      dtype='object')]

  ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


In [8]:
print(timelogs)

[75.20939755439758, 75.69101405143738, 74.95247769355774, 75.63265204429626, 75.64499044418335, 75.17832660675049, 75.41127276420593, 75.7157199382782, 75.62274289131165, 75.583247423172, 75.58476424217224, 75.24678087234497, 75.29982781410217, 74.89348554611206, 74.85167169570923, 75.39219188690186, 74.58147072792053, 74.1956398487091, 74.33109021186829, 74.65211176872253, 74.24645662307739, 74.1836793422699, 74.79843521118164, 74.72419881820679, 75.01094460487366, 75.15067338943481, 75.24850082397461, 75.27187037467957, 74.90040349960327, 75.34499645233154, 75.16779637336731, 74.44896531105042, 74.26818609237671, 74.6088936328888, 74.2044792175293, 74.80146741867065, 74.58861708641052, 74.02318692207336, 74.5551609992981, 75.07039904594421, 74.0175883769989, 74.3556752204895, 74.91946792602539, 74.37077116966248, 74.66437029838562, 75.22838830947876, 75.02461695671082, 75.19545602798462, 74.61784434318542, 74.63368892669678]
